In [1]:
import sys, os; sys.path.insert(0, '..')
import random, logging, math
import pyzx as zx
from fractions import Fraction
from pyzx.gadget_extract import *
Z = zx.VertexType.Z
X = zx.VertexType.X
SE = zx.EdgeType.SIMPLE
HE = zx.EdgeType.HADAMARD
# zx.settings.drawing_backend = 'matplotlib'

In [2]:
c = zx.qasm("""
qreg q[4];
cx q[0], q[1];
cx q[1], q[2];
s q[2];
""")
g = c.to_graph()
print(g.scalar)
print(g.num_edges() - g.num_vertices() + g.num_outputs())

2.00+0.00i = sqrt(2)^2
2


In [3]:
def num_spiders(g):
    return len([v for v in g.vertices() if g.type(v) == Z or g.type(v) == X])

def inv(g):
    return (g.scalar.power2 == g.num_edges() - num_spiders(g) - g.num_outputs())

In [4]:
c = zx.Circuit.load('../circuits/QFT_and_Adders/QFTAdd16_before')
g = c.to_graph()
zx.full_reduce(g)
print(g.num_edges() - g.num_vertices() + g.num_outputs())
print(g.scalar.power2 == g.num_edges() - num_spiders(g) - g.num_outputs())

559
True


In [5]:
zx.draw(c)

In [7]:
def squash_reduce(g, err_budget, quiet=True):
    total_err = 0
    for i in range(50, -1, -1):
        zx.full_reduce(g)
        th = Fraction(1, 2**i)
        if not quiet: print(f'threshold Δθ <= 2^-{i} π')
        for v in list(g.vertices()):
            p = g.phase(v)
            if p.denominator > 2:
                new_p = Fraction(round(2*p), 2)
                err = abs(float(p)-round(new_p))
                if err <= th:
                    if total_err + err > err_budget:
                        zx.full_reduce(g)
                        if not quiet: print('exceeded error budget, done.')
                        return
                    else:
                        norm_err = 2*(math.sin(err/2)**2)
                        if not quiet: print(f' * {p} ~ {new_p} (ε = {norm_err})')
                        g.set_phase(v, new_p)
                        total_err += norm_err
    zx.full_reduce(g)

In [8]:
print(c.to_basic_gates().stats())

Circuit QFTAdd16_before on 32 qubits with 1822 gates.
        1026 is the T-count
        796 Cliffords among which
        716 2-qubit gates (716 CNOT, 0 other) and
        32 Hadamard gates.


In [9]:
g1 = c.to_graph()
squash_reduce(g1, err_budget=0.0)
print("extracting circuit...")
c1 = zx.extract_circuit(g1, up_to_perm=True)
zx.basic_optimization(c1)
#zx.draw(c1)
print(c1.stats())

extracting circuit...
Circuit  on 32 qubits with 2142 gates.
        402 is the T-count
        1740 Cliffords among which
        964 2-qubit gates (613 CNOT, 351 other) and
        776 Hadamard gates.


In [41]:
g1 = c.to_graph()
squash_reduce(g1, err_budget=2**-15)
print("extracting circuit...")
c1 = zx.extract_circuit(g1, up_to_perm=True)
zx.basic_optimization(c1)
#zx.draw(c1)
print(c1.stats())

extracting circuit...
Circuit  on 32 qubits with 2142 gates.
        402 is the T-count
        1740 Cliffords among which
        964 2-qubit gates (613 CNOT, 351 other) and
        776 Hadamard gates.


In [10]:
g1 = c.to_graph()
squash_reduce(g1, err_budget=0.2)
print("extracting circuit...")
c1 = zx.extract_circuit(g1, up_to_perm=True)
#zx.draw(c1)
print(c1.stats())

extracting circuit...
Circuit  on 32 qubits with 680 gates.
        122 is the T-count
        558 Cliffords among which
        308 2-qubit gates (121 CNOT, 187 other) and
        244 Hadamard gates.
